# Neural Network for Cat Classification

In [29]:
# Install necessary libraries
!pip install --upgrade pip
!pip install tensorflow==2.18.0
!pip install scikit-learn
!pip install imbalanced-learn
!pip install pandas --no-cache-dir
!pip install numpy
!pip install matplotlib
!pip install jupyter
!pip install openpyxl
!pip install seaborn

In [30]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.utils.class_weight import compute_class_weight

## Step 1: Data Preprocessing

In [31]:
def preprocess_data(data):
    """
    Preprocess the data by handling missing values, encoding categorical variables,
    and scaling numeric features.
    """
    # Create a copy to avoid modifying the original data
    df = data.copy()
    
    # Convert 'Gender' to binary (F=0, M=1)
    df['Gender'] = df['Gender'].map({'F': 0, 'M': 1})
    
    # Label encode categorical variables
    categorical_cols = ['Age', 'HousingType', 'Zone']
    label_encoders = {}
    for col in categorical_cols:
        label_encoders[col] = LabelEncoder()
        df[col] = label_encoders[col].fit_transform(df[col])
    
    # Handle missing values in numeric columns
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    imputer = SimpleImputer(strategy='mean')
    df[numeric_cols] = imputer.fit_transform(df[numeric_cols])
    
    # Label encode the target variable
    label_encoder_race = LabelEncoder()
    df['Race'] = label_encoder_race.fit_transform(df['Race'])
    
    return df, label_encoders, label_encoder_race

# Load and preprocess data
data = pd.read_excel('Dataset (2).xlsx', sheet_name='Data')
processed_data, label_encoders, label_encoder_race = preprocess_data(data)

# Split features and target
X = processed_data.drop(columns=['Race'])
y = processed_data['Race']

# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("Training data shape:", X_train.shape)
print("Testing data shape:", X_test.shape)
print("Number of classes:", len(np.unique(y)))

ValueError: could not convert string to float: 'Unknown'

## Step 2: Model definition

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(len(np.unique(y)), activation='softmax')
])

## Step 3: Training setup

In [ ]:
# Convert targets to numpy array
y_train = np.array(y_train)
y_test = np.array(y_test)

# Compute balanced class weights
unique_classes = np.unique(y_train)
class_weights = compute_class_weight('balanced', classes=unique_classes, y=y_train)
class_weight_dict = dict(zip(unique_classes, class_weights))

# Learning rate schedule
initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=1000, decay_rate=0.9
)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=15, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.2, patience=5),
]

# Step 4: Training

In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=200,
    batch_size=32,
    validation_split=0.2,
    class_weight=class_weight_dict,
    callbacks=callbacks,
    verbose=1
)

# Step 5: Evaluation

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"\nTest accuracy: {test_accuracy:.4f}")